# Project 1
## 1.
Objective: $Min \sum_{t=1}^{T} f_ty_t + \sum_{t=1}^{T} c_tx_t + \sum_{t=1}^{T} h_ts_t$

Subject to:
$d_t = x_t + s_{t-1} - s_t \qquad \forall t \in H$
$x_t \leq D_ty_t \qquad \forall t \in H$

$y_t \in [0,1]$
<br>
$x_t, s_t \geq 0$

Where:
$D_t = \sum_{i=t}^{T} d_i$
$s_0, s_T = 0$

In [148]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

# Import data and define constants

T = 24  # Num periods
rep = 1  # Tab number

tabs = {'6-1': '6-periods (1)',
    '6-2': '6-periods (2)',
    '12-1': '12-periods (1)',
    '12-2': '12-periods (2)',
    '24-1': '24-periods (1)',
    '24-2': '24-periods (2)',
    '52-1': '52-periods (1)',
    '52-2': '52-periods (2)',
    '104-1': '104-periods (1)',
    '104-2': '104-periods (2)'}

xls = pd.ExcelFile(r'../ULSP-instancesR.xlsx')

df = pd.read_excel(xls, sheet_name=tabs[f'{T}-{rep}'])

# Add zero row and adjust index to align with indicies prompt
df.loc[-1] = [0]*6
df.index = df.index + 1
df = df.sort_index()

# Constants
d = df['Demand Forecast']
f = df['Setup Cost']
c = df['Production cost']
h = df['Holding cost']
b = df['Backlogging cost']

D = [d[i:].sum() for i in range(len(d))]

# Set
H = range(1,T+1)
H_0 = range(T+1)

# Gurobi environment
ENV = gp.Env()
ENV.setParam('OutputFlag', 0)


Restricted license - for non-production use only - expires 2025-11-24


In [149]:
def extract_model_vars(x, y, s):
    # ---- Extract variable values ----
    x_values = np.zeros(len(H_0))
    y_values = np.zeros(len(H_0))
    s_values = np.zeros(len(H_0))

    for t in H:
        x_values[t] = x[t].X
        y_values[t] = y[t].X
        s_values[t] = s[t].X

    return x_values, y_values, s_values


def inequality_separation(x, y, s):
    S = {}
    v = {}
    Djl = {}
    for l in H:
        S[l] = set()
        for j in range(1,l+1):
            Djl[(j,l)] = d[j:l+1].sum()
            if x[j] - Djl[(j,l)]*y[j] > 0:
                S[l].add(j)
        v[l] = sum([x[t] - Djl[(t,l)]*y[t] for t in S[l]]) - s[l] # TODO: confirm, error in problem 5, summation on 3rd line should be j \in S_l, not t \in S_l?
    return v, S, Djl


def run_model(relax=False, isp=False):
    start_time = time.time()

    # ---- Initiate model ----
    model = gp.Model(env=ENV)

    # ---- Decision variables ----

    # Units produced
    x = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='x')

    # Whether a lot is made
    if relax:
        y = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='y')
    else:
        y = model.addVars(T+1, vtype=GRB.BINARY, name='y')

    # Amount held in inventory at end of t
    s = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='s')


    # ---- Constraints ----

    # Initial values
    model.addConstr(s[0] == 0)
    model.addConstr(x[0] == 0)
    model.addConstr(y[0] == 0)

    # Material balance
    model.addConstrs(int(d[t]) == x[t] + s[t-1] - s[t] for t in H)

    # y definition
    model.addConstrs(x[t] <= D[t]*y[t] for t in H)


    # ---- Objective ----
    obj = gp.LinExpr()

    for t in H:
        obj.addTerms([f[t], c[t], h[t]], [y[t], x[t], s[t]])

    model.setObjective(obj, GRB.MINIMIZE)

    # ---- Optimize ----
    model.optimize()

    # ---- Extract values ----
    x_values, y_values, s_values = extract_model_vars(x, y, s)

    if isp:
        if not relax:
            print("WARNING: isp is only designed to be used when relax=True")

        v, S, Djl = inequality_separation(x_values, y_values, s_values)

        limit = 0
        count = 0
        max_count = 10000

        while max(v.values())>1e-10 and count <= max_count:
            # l_max = max(v.items(), key=lambda x: x[1])[0]
            # model.addConstrs(gp.quicksum(x[t] for t in S[l]) <= gp.quicksum(Djl[(t,l)]*y[t] for t in S[l]) + s[l]for l in range(1,l_max+1))
            print("v: ", v)
            l = max(v.items(), key=lambda x: x[1])[0]
            print("v_max: ", max(v.items(), key=lambda x: x[1]))
            model.addConstr(gp.quicksum(x[t] for t in S[l]) <= gp.quicksum(Djl[(t,l)]*y[t] for t in S[l]) + s[l])
            model.optimize()
            x_values, y_values, s_values = extract_model_vars(x, y, s)

            v, S, Djl = inequality_separation(x_values, y_values, s_values)

            print("final v: ", v)
            print(f"Count: {count}")

            if count == max_count-1:
                print("WARNING: isp max iterations exceeded")
            count += 1

    end_time = time.time()
    total_time = end_time - start_time

    return x_values, y_values, s_values, total_time

def print_model_results(relax, isp):
    x_values, y_values, s_values, total_time = run_model(relax=relax, isp=isp)
    print("x: ", x_values)
    print("y: ", y_values)
    print("s: ", s_values)
    print("total_time: ", total_time)

print("MILP model without relaxation:")
print_model_results(relax=False, isp=False)

print()
print("LP model with relaxation:")
print_model_results(relax=True, isp=False)

MILP model without relaxation:
x:  [  0. 660. 837. 164.   0. 857. 190.   0.   0. 100. 454. 924. 496. 703.
 469. 706. 907. 321.   0. 192.   0.   0. 883. 783. 727.]
y:  [0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1.
 1.]
s:  [ 0.  0.  0.  0.  0.  0. 73.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]
total_time:  0.006085634231567383

LP model with relaxation:
x:  [  0. 660. 837. 164.   0. 857. 117.  73.   0. 100. 454. 924. 496. 703.
 469. 706. 907. 321.   0. 192.   0.   0. 883. 783. 727.]
y:  [0.         0.06362672 0.08617317 0.01847679 0.         0.09837006
 0.01489497 0.00943396 0.         0.01304631 0.06001322 0.12993953
 0.08016809 0.12352838 0.09402566 0.15622925 0.23787044 0.11046111
 0.         0.07427466 0.         0.         0.3689929  0.51854305
 1.        ]
s:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
total_time:  0.0020570755004882812


# 2.
Computation times are comparable at around 0.001s for the 6-element dataset and 0.006 for the 104-element dataset. Solutions for production quantities and holding are the same for the 6-element set, but not the 104-element set, although the y variable indicating whether a lot is produced is no longer strictly 0 or 1. If it is 0, no lot is produced, but when a lot is produced, y is a fractional value. This will not affect the production plan, but it will give an artificially lower objective.

# 3.
 Yes, we could use a heuristic here (i.e. dynamic programming) to get the same result as the MILP. In fact, this would be a good application for the Silver-Meal heuristic, which is similar to (Wagner-Whitin, 1958), except the former calculates cost per unit produced, and (Wagner-Whitin, 1958) looks at the absolute cost to fill the demand in a particular period.

Pattern: Demand for each period is filled from only one source, i.e. all demand is satisfied by production or inventory, for each period.

In [150]:
# TODO
# See https://www.youtube.com/watch?v=7vE-gm9qxpk

z = np.zeros(T+1)
x_ww = np.zeros(T+1)

# Not complete
for t in H:
    t_t = t
    z_t = np.zeros(t+1)
    while t_t > 0:

        z_t[t_t] = f[t_t]
        t_t -= 1

    z[t] = min(z_t)


# 4.
This inequality states that the amount of units produced in any subset of periods must be less than or equal to the demand of those periods plus the ending inventory.
Indeed, the constraint for the definition of $y_t$ is:
$x_t \leq D_ty_t \qquad \forall t \in H$

If this is the case, then cumulative (i.e. summation of) $x_t$ must also be less than the cumulative $Dy_t$. $s_l$ is defined to be nonnegative, so adding this to the right side does not change the inequality.

# 5.

See inequality_separation() function in cell 3. Example output is shown below.

In [151]:
x, y, s, _ = run_model(relax=True)
v, S, Djl = inequality_separation(x, y, s)


print(v)
print(max(v.items(), key=lambda x: x[1]))


{1: 618.0063626723224, 2: 1329.6238520621366, 3: 1466.0264757960042, 4: 1466.0264757960042, 5: 2094.5102120497722, 6: 2178.5698307881776, 7: 2230.3286059694215, 8: 2230.3286059694215, 9: 2299.9264062829316, 10: 2588.6544183843926, 11: 3056.2217531351757, 12: 3267.4469043304166, 13: 3479.9840377438263, 14: 3577.678200995259, 15: 3614.442350164843, 16: 3445.924854334916, 17: 3350.826135050157, 18: 3350.826135050157, 19: 3279.6839230851165, 20: 3279.6839230851165, 21: 3279.6839230851165, 22: 2626.683127395957, 23: 1641.6156514018946, 24: 2.2737367544323206e-13}
(15, 3614.442350164843)


# 6.
Applying the inequality separation procedure in Gurobi.


In [152]:
print_model_results(relax=True, isp=True)



v:  {1: 618.0063626723224, 2: 1329.6238520621366, 3: 1466.0264757960042, 4: 1466.0264757960042, 5: 2094.5102120497722, 6: 2178.5698307881776, 7: 2230.3286059694215, 8: 2230.3286059694215, 9: 2299.9264062829316, 10: 2588.6544183843926, 11: 3056.2217531351757, 12: 3267.4469043304166, 13: 3479.9840377438263, 14: 3577.678200995259, 15: 3614.442350164843, 16: 3445.924854334916, 17: 3350.826135050157, 18: 3350.826135050157, 19: 3279.6839230851165, 20: 3279.6839230851165, 21: 3279.6839230851165, 22: 2626.683127395957, 23: 1641.6156514018946, 24: 2.2737367544323206e-13}
v_max:  (15, 3614.442350164843)
final v:  {1: 254.35819939354246, 2: 764.8730567280963, 3: 911.7104630706597, 4: 911.7104630706597, 5: 1594.7223011271847, 6: 1686.2262464827695, 7: 1742.629772459347, 8: 1742.629772459347, 9: 1818.590245095233, 10: 2136.204789540278, 11: 2662.56321654981, 12: 2905.3472224640327, 13: 3162.6139423037416, 14: 3290.1490387471144, 15: 3371.833654512669, 16: 3261.0255966466866, 17: 3186.3510555167527,

# 7.
New formulation:

$Min \sum_{t=1}^{T} f_ty_t + \sum_{t=1}^{T}\sum_{q=1}^{t} c_qd_tw_{q,t} + \sum_{t=1}^T\sum_{q=1}^{t-1}\sum_{i=q}^{t-1}h_id_tw_{q,t}$
<br>
(Setup cost + unit production costs + holding cost of inventory)

Subject to:
$\sum_{q=1}^{t}w_{q,t} = 1 \qquad  \forall t \in T$     (Ensures demand is met)

$w_{q,t} \leq y_q \qquad  \forall q \in T \quad \forall t \in T$       (Defines y as 1 if any products are produced, 0 if not)
<br>
$y_t \geq 0$
<br>
$0 \leq w_{q,t} \leq 1$

Where:
$T_q = T-q$ (Remaining periods)

# 8.


In [153]:
# Tq = [T+1-i for i in range(T+1)]

def fplr_model(relax=False):

    start_time = time.time()

    # ---- Initiate model ----
    model = gp.Model(env=ENV)

    # ---- Decision variables ----

    # Units produced
    w = model.addVars(T+1, T+1, vtype=GRB.CONTINUOUS, name='w')

    # Whether a lot is made
    if relax:
        y = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='y')
    else:
        y = model.addVars(T+1, vtype=GRB.BINARY, name='y')


    # ---- Constraints ----

    # Ensure demand is met
    model.addConstrs(gp.quicksum(w[q,t] for q in range(1,t+1)) == 1 for t in H)

    # Define setup variable y
    # model.addConstrs(gp.quicksum(w[q,t] for t in range(q,T+1)) <= y[q]*Tq[q] for q in H)  # Can use this constraint if y is not relaxed
    model.addConstrs(w[q,t] <= y[q] for q in H for t in range(q,T+1))


    # ---- Objective ----
    obj = gp.LinExpr()

    for t in H:
        obj.addTerms(f[t], y[t])
        for q in range(1,t+1):
            obj.addTerms(c[q]*d[t], w[q,t])
        # for i in range (1,t):
        #     for q in range(1,i+1):
        #         obj.addTerms(h[t]*d[i], w[q,i])
    for t in range(1,T+1):
        for q in range(1,t):
            obj.addTerms(h[q:t].sum()*d[t], w[q,t])


    model.setObjective(obj, GRB.MINIMIZE)

    # ---- Optimize ----
    model.optimize()

    # ---- Extract variable values ----
    w_values = np.zeros((len(H_0),len(H_0)))
    y_values = np.zeros(len(H_0))

    for q in H:
        for t in H:
            w_values[q,t] = w[q,t].X
        y_values[q] = y[q].X

    end_time = time.time()
    total_time = end_time - start_time

    return w_values, y_values, total_time

w_values, y_values, total_time = fplr_model(relax=True)

units_per_period = np.zeros(len(H_0))

for q in H_0:
    units_per_period[q] = np.sum([w_values[q,t]*d[t] for t in H_0])

print("w: ", w_values)
print("y: ", y_values)
print("total_time: ", total_time)
print("units_per_period: ", units_per_period)


w:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0.
  0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

# 9.
Objective: $Min \sum_{t=1}^{T} f_ty_t + \sum_{t=1}^{T} c_tx_t + \sum_{t=1}^{T} h_ts_t + \sum_{t=1}^{T} b_tr_t$

Subject to:
$d_t = x_t + s_{t-1} - s_t - r_t + r_{t-1} \qquad \forall t \in H$
$x_t \leq Dy_t \qquad \forall t \in H$

$y_t \in [0,1]$
<br>
$x_t, s_t, r_t \geq 0$

Where:
$D = \sum_{i=0}^{T} d_i$
$s_0, s_T, r_0, r_T = 0$

In [154]:
def extract_model_vars_backlog(x, y, s, r):
    # ---- Extract variable values ----
    x_values = np.zeros(len(H_0))
    y_values = np.zeros(len(H_0))
    s_values = np.zeros(len(H_0))
    r_values = np.zeros(len(H_0))

    for t in H:
        x_values[t] = x[t].X
        y_values[t] = y[t].X
        s_values[t] = s[t].X
        r_values[t] = r[t].X

    return x_values, y_values, s_values, r_values


def run_model_backlog(relax=False, isp=False):
    start_time = time.time()

    # ---- Initiate model ----
    model = gp.Model(env=ENV)

    # ---- Decision variables ----

    # Units produced
    x = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='x')

    # Whether a lot is made
    if relax:
        y = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='y')
    else:
        y = model.addVars(T+1, vtype=GRB.BINARY, name='y')

    # Amount held in inventory at end of t
    s = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='s')

    # Backlog at end of t
    r = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='r')

    # ---- Constraints ----

    # Initial values
    model.addConstr(s[0] == 0)
    model.addConstr(x[0] == 0)
    model.addConstr(y[0] == 0)
    model.addConstr(r[0] == 0)
    model.addConstr(r[T] == 0)

    # Material balance
    model.addConstrs(int(d[t]) == x[t] + s[t-1] - s[t] + r[t] - r[t-1] for t in H)

    # y definition
    model.addConstrs(x[t] <= D[0]*y[t] for t in H)

    # ---- Objective ----
    obj = gp.LinExpr()

    for t in H:
        obj.addTerms([f[t], c[t], h[t], b[t]], [y[t], x[t], s[t], r[t]])

    model.setObjective(obj, GRB.MINIMIZE)

    # ---- Optimize ----
    model.optimize()

    # ---- Extract values ----
    x_values, y_values, s_values, r_values = extract_model_vars_backlog(x, y, s, r)

    if isp:
        if not relax:
            print("WARNING: isp is only designed to be used when relax=True")

        v, S, Djl = inequality_separation(x_values, y_values, s_values)

        count = 0
        while max(v.values())>0 and count <= 100:
            # l_max = max(v.items(), key=lambda x: x[1])[0]
            # model.addConstrs(gp.quicksum(x[t] for t in S[l]) <= gp.quicksum(Djl[(t,l)]*y[t] for t in S[l]) + s[l]for l in range(1,l_max+1))
            print("v: ", v)
            l = max(v.items(), key=lambda x: x[1])[0]
            print("l: ", l)
            print("v_max: ", max(v.items(), key=lambda x: x[1]))
            model.addConstr(gp.quicksum(x[t] for t in S[l]) <= gp.quicksum(Djl[(t,l)]*y[t] for t in S[l]) + s[l])
            model.optimize()
            x_values, y_values, s_values = extract_model_vars(x, y, s)

            v, S, Djl = inequality_separation(x_values, y_values, s_values)

            print("final v: ", v)

            if count == 99:
                print("WARNING: isp max iterations exceeded")
            count += 1

    end_time = time.time()
    total_time = end_time - start_time

    return x_values, y_values, s_values, r_values, total_time

def print_model_results_backlog(relax, isp):
    x_values, y_values, s_values, r_values, total_time = run_model_backlog(relax=relax, isp=isp)
    print("x: ", x_values)
    print("y: ", y_values)
    print("s: ", s_values)
    print("r: ", r_values)
    print("total_time: ", total_time)

print("MILP model, with backlog, without relaxation:")
print_model_results_backlog(relax=False, isp=False)

print()
print("LP model, with backlog, with relaxation:")
print_model_results_backlog(relax=True, isp=False)

MILP model, with backlog, without relaxation:
x:  [  0. 660. 837. 164.   0. 857. 190.   0.   0. 100. 454. 924. 496. 703.
 469. 706. 907. 321.   0. 192.   0.   0. 883. 783. 727.]
y:  [ 0.  1.  1.  1. -0.  1.  1.  0. -0.  1.  1.  1.  1.  1.  1.  1.  1.  1.
 -0.  1. -0.  0.  1.  1.  1.]
s:  [ 0.  0.  0.  0.  0.  0. 73.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]
r:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
total_time:  0.007086753845214844

LP model, with backlog, with relaxation:
x:  [  0. 660. 837. 164.   0. 857. 117.  73.   0. 100. 454. 924. 496. 703.
 469. 706. 907. 321.   0. 192.   0.   0. 883. 783. 727.]
y:  [0.         0.06362672 0.08069025 0.01581028 0.         0.08261834
 0.01127928 0.0070375  0.         0.00964041 0.04376747 0.08907741
 0.04781645 0.0677721  0.04521354 0.06806131 0.08743854 0.03094572
 0.         0.01850959 0.         0.         0.08512484 0.07548443
 0.0700858 ]
s:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0

# 10.
Pattern: the demand for each period is satisfied by one, and only one, source. Said another way, the demand for each period is satisfied completely by either production, inventory, or the backlog.

In [155]:
# TODO: not complete yet

def calculate_optimal_schedule():

    optimal_cost = [0] * (T + 1)
    production_decision = [0] * (T + 1)

    # Calculate optimal costs
    for i in range(1, T + 1):
        min_cost = float('inf')
        for j in range(1, i + 1):
            cost = f[j]+sum([d[k]*c[k] for k in range(j,j+1)]) + sum(h[j:i]) + optimal_cost[j-1]
            print(f"cost at i={i}, j={j}: {cost}")
            if cost < min_cost:
                min_cost = cost
                production_decision[i] = j
                print(f"production decision[i] = {j}")
        optimal_cost[i] = min_cost
        print(f"optimal_cost at i = {i}: {optimal_cost}")

    # Backtrack to find the optimal production schedule
    schedule = []
    i = T
    while i > 0:
        schedule.append(i - production_decision[i] + 1)
        i = production_decision[i] - 1
        print(f"i: {i}")
        print(f"schedule: {schedule}")

    return optimal_cost[T], schedule[::-1]

# Calculate optimal schedule
cost, schedule = calculate_optimal_schedule()
print("Optimal cost:", cost)
print("Optimal production schedule:", schedule)


cost at i=1, j=1: 3512
production decision[i] = 1
optimal_cost at i = 1: [0, 3512, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
cost at i=2, j=1: 3520
production decision[i] = 1
cost at i=2, j=2: 10468
optimal_cost at i = 2: [0, 3512, 3520, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
cost at i=3, j=1: 3529
production decision[i] = 1
cost at i=3, j=2: 10477
cost at i=3, j=3: 5160
optimal_cost at i = 3: [0, 3512, 3520, 3529, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
cost at i=4, j=1: 3535
production decision[i] = 1
cost at i=4, j=2: 10483
cost at i=4, j=3: 5166
cost at i=4, j=4: 4026
optimal_cost at i = 4: [0, 3512, 3520, 3529, 3535, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
cost at i=5, j=1: 3541
production decision[i] = 1
cost at i=5, j=2: 10489
cost at i=5, j=3: 5172
cost at i=5, j=4: 4032
cost at i=5, j=5: 9000
optimal_cost at i = 5: [0, 3512, 3520, 3529, 3535, 3541, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 